In [1]:
import wapi
import pytz
import pandas as pd
import plotly.offline as offline
import plotly.graph_objs as go
import plotly.tools as tls
from datetime import datetime, timedelta

session = wapi.Session(client_id='client_id', client_secret='client_secret')
offline.offline.init_notebook_mode(connected=True)

meta_criteria = {'commodity': 'POW',
                 'category': ['WND', 'Intraday'], 
                 'unit': 'MWH/H',
                 'area': 'DK1', 
                 'data_type': ['A', 'F']}
curves = session.search(**meta_criteria)
[c.name for c in curves]

['pro dk1 wnd intraday ec00da 01 mwh/h cet min15 f',
 'pro dk1 wnd intraday lastec 01 mwh/h cet min15 f',
 'pro dk1 wnd intraday tso 01 mwh/h cet min15 f',
 'pro dk1 wnd intraday 01 mwh/h cet min15 a',
 'pro dk1 wnd intraday ec00da mwh/h cet min15 f',
 'pro dk1 wnd intraday lastec mwh/h cet min15 f',
 'pro dk1 wnd intraday tso mwh/h cet min15 f',
 'pro dk1 wnd intraday mwh/h cet min15 a',
 'pro dk1 wnd ec00 01 mwh/h cet min15 f',
 'pro dk1 wnd ec00 mwh/h cet min15 f',
 'pro dk1 wnd ec12 01 mwh/h cet min15 f',
 'pro dk1 wnd ec12 mwh/h cet min15 f',
 'pro dk1 wnd gfs00 01 mwh/h cet min15 f',
 'pro dk1 wnd gfs00 mwh/h cet min15 f',
 'pro dk1 wnd gfs06 01 mwh/h cet min15 f',
 'pro dk1 wnd gfs06 mwh/h cet min15 f',
 'pro dk1 wnd gfs12 01 mwh/h cet min15 f',
 'pro dk1 wnd gfs12 mwh/h cet min15 f',
 'pro dk1 wnd gfs18 01 mwh/h cet min15 f',
 'pro dk1 wnd gfs18 mwh/h cet min15 f',
 'pro dk1 wnd ec00ens 01 mwh/h cet min15 f',
 'pro dk1 wnd ec00ens mwh/h cet min15 f',
 'pro dk1 wnd ec12ens 01 mw

In [2]:
intraday_curves = [c for c in curves if 'INTRADAY' in c.categories]
data = {}
label = {}
issue_date = pytz.timezone('CET').localize(datetime.strptime('2018-04-12 00:00', "%Y-%m-%d %H:%M"))
date_from = issue_date + timedelta(minutes=15)
date_to = date_from + timedelta(hours=24)
for c in intraday_curves:
    if c.curve_type == 'INSTANCES':
        ts = c.get_instance(issue_date)
        if ts is not None:
            label[c.name] = ';'.join(c.sources)
            data[c.name] = ts.to_pandas()
    elif c.curve_type == 'TIME_SERIES':
        data[c.name] = c.get_data(data_from=date_from, 
                                  data_to=date_to).to_pandas()
        label[c.name] = 'Actual'

fig = tls.make_subplots(len(data), cols=1, shared_xaxes=True)
for key in data.keys():
    ts = data[key]
    fig.append_trace({'x': ts.index, 'y': ts.values, 'type': 'scatter', 
                      'name': label[key]}, 1, 1)

title = ' '.join([c.commodity,
                  ' '.join(c.categories),
                  c.area,
                  issue_date.strftime("%Y-%m-%d %H:%M"),
                  c.time_zone])
fig['layout'].update(title=title)
fig['layout'].update(yaxis=dict(title=c.unit))    
offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]
[ (6,1) x1,y6 ]



In [3]:
curve = intraday_curves[0]
latest_instance = curve.get_latest(with_data=False)
issue_date_str = latest_instance.issue_date
issue_date = latest_instance.tz.localize(
    datetime.strptime(latest_instance.issue_date, 
                      "%Y-%m-%dT%H:%M:%SZ"))
value_date = issue_date + timedelta(minutes=15)
index = []
values = []
forecast_exists = True
while forecast_exists:
    ts = curve.get_instance(issue_date=issue_date)
    try:
        ts = curve.get_instance(issue_date=issue_date).to_pandas()
        values.append(ts[value_date])
        index.append(issue_date)
        issue_date = issue_date - timedelta(minutes=15)
    except:
        forecast_exists = False
    
fig = tls.make_subplots(rows=1, cols=1, shared_xaxes=True)
fig.append_trace({'x': index, 'y': values, 'type': 'scatter'}, 1, 1)

fig['layout'].update(yaxis=dict(title=curve.unit))    

title = ' '.join([' '.join(curve.categories), 
                  curve.area, 
                  curve.time_zone, 
                  'contract development for', 
                  value_date.strftime("%Y-%m-%d %H:%M"), 
                  str(value_date.tzinfo)])
fig['layout'].update(title=title)
offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [4]:
curves_at00 = [c for c in curves if 'INTRADAY' not in c.categories and (c.data_type == 'A' or c.sources [0] in ["EC00", "EC00Ens", 'GFS00'] )]
[c.name for c in curves_at00]
        

['pro dk1 wnd ec00 01 mwh/h cet min15 f',
 'pro dk1 wnd ec00 mwh/h cet min15 f',
 'pro dk1 wnd gfs00 01 mwh/h cet min15 f',
 'pro dk1 wnd gfs00 mwh/h cet min15 f',
 'pro dk1 wnd ec00ens 01 mwh/h cet min15 f',
 'pro dk1 wnd ec00ens mwh/h cet min15 f',
 'pro dk1 wnd 01 mwh/h cet min15 a',
 'pro dk1 wnd mwh/h cet min15 a']

In [5]:
data = {}
label = {}
issue_date = pytz.timezone('CET').localize(datetime.strptime('2018-04-12 00:00', "%Y-%m-%d %H:%M"))
date_to = date_from + timedelta(days=14)
for c in curves_at00:
    ts = None
    if c.curve_type == 'TAGGED_INSTANCES':
        ts = c.get_instance(issue_date=issue_date, tag='Avg')
        label[c.name] = ' '.join([';'.join(c.sources), 'Avg'])
    elif c.curve_type == 'INSTANCES':
        ts = c.get_instance(issue_date=issue_date)
        label[c.name] = ';'.join(c.sources)
    elif c.curve_type == 'TIME_SERIES':
        data[c.name] = c.get_data(data_from=date_from, data_to=date_to).to_pandas()
        label[c.name] = 'Actual'
    if ts is not None:
        data[c.name] = ts.to_pandas()
        
fig = tls.make_subplots(len(data), cols=1, shared_xaxes=True)
for key in data.keys():
    ts = data[key]
    fig.append_trace({'x': ts.index, 'y': ts.values, 'type': 'scatter', 
                      'name': label[key]}, 1, 1)

title = ' '.join([c.commodity,
                  ' '.join(c.categories),
                  c.area,
                  issue_date.strftime("%Y-%m-%d %H:%M"),
                  c.time_zone])
                  
fig['layout'].update(title=title)
fig['layout'].update(yaxis=dict(title=c.unit))    
offline.iplot(fig)  

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]
[ (6,1) x1,y6 ]

